# Comparativo de novos casos por dia por estado [in Portuguese]
> Casos confirmados e mortes - novos casos por dia (linear scale)

- toc: false
- badges: true
- comments: true
- author: Cleber Jorge Amaral
- categories: [covid-19, brasil, comparativo, altair, jupyter]
- image: images/brazil-states-linear-cases-and-deaths-newcases.png

### NOTA
Os dados utilizados nestas análises são inerentemente incertos. Não há garantia de que os dados e as análises estejam devidamente atualizados e corretos. Antes de tomar conclusões é fundamental verificar dados e conclusões com outras fontes.

In [1]:
#hide
import pandas as pd
import altair as alt
import math
import requests
from altair_saver import save
from IPython.display import HTML

CHART_WIDTH = 600
CHART_HEIGHT = 400

/usr/local/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
#hide
STATE_COLUMN = "Estado"
DATE_COLUMN = "Data"
DEATH_COLUMN = "Obitos"
NEWCONFIRMED_CASES = "Confirmado"
NEWDEATH_COLUMN = "Obito"
NEWRECOVERY_COLUMN = "Recuperado"
ACTIVE_COLUMN = "Ativo"
TOTAL_BY_STATE = "total por estado"
CONFIRMED_CASES = 'Casos confirmados'
DEATHS = 'Mortes'
NEW_CASES = 'novos casos por dia'
ACCUMULATED = 'acumulado'

In [3]:
#hide
#url = ('https://data.brasil.io/dataset/covid19/caso_full.csv.gz')
url = ('https://github.com/cleberjamaral/fastpages/raw/master/assets/db/caso.csv.gz')

# Pretending to be a browser
header = {
  "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/581.0.4044.138 Safari/537.36",
  "x-requested-with": "XMLHttpRequest",
  "accept-encoding": "gzip, deflate, br"
}

response = requests.get(url, headers=header, stream=True)

raw = response.raw
with open('caso_full.csv.gz', 'wb') as out_file:
    while True:
        chunk = raw.read(1024, decode_content=True)
        if not chunk:
            break
        out_file.write(chunk)

src = pd.read_csv('caso_full.csv.gz')

In [4]:
#hide
df = src[(src['place_type'] == 'state')]
df = df.rename(columns={'date': DATE_COLUMN, 'state': STATE_COLUMN, 'confirmed': NEWCONFIRMED_CASES, 'deaths': NEWDEATH_COLUMN})
#df.head()

,Data,Estado,city,place_type,Confirmado,Obito,order_for_place,is_last,estimated_population_2019,estimated_population,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-10-10,AP,NaN,state,49303,725,204,True,845731.0,861773.0,16.0,5721.11217,0.0147
1,2020-10-09,AP,NaN,state,49215,725,203,False,845731.0,861773.0,16.0,5710.90067,0.0147
2,2020-10-08,AP,NaN,state,49085,724,202,False,845731.0,861773.0,16.0,5695.81549,0.0147
3,2020-10-07,AP,NaN,state,48963,722,201,False,845731.0,861773.0,16.0,5681.65863,0.0147
4,2020-10-06,AP,NaN,state,48814,721,200,False,845731.0,861773.0,16.0,5664.36869,0.0148


In [5]:
#hide
df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN])
df[DATE_COLUMN] = df[DATE_COLUMN].dt.strftime('%m/%d/%y')

In [6]:
#hide
LAST_DATE = max(df[DATE_COLUMN])

In [7]:
#hide_input
alt.data_transformers.disable_max_rows()
opt = df[STATE_COLUMN].unique()
opt.sort()
input_dropdown = alt.binding_select(options=opt)
selection1 = alt.selection_single(fields=[STATE_COLUMN], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=[STATE_COLUMN], on='mouseover')
color = alt.condition(selection1 | selection2,
                    alt.Color(STATE_COLUMN+':N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

chart = alt.Chart(df).mark_bar().encode(
    x=alt.X(DATE_COLUMN+':O', axis=alt.Axis(title=DATE_COLUMN)),
    y=alt.Y(NEWCONFIRMED_CASES+':Q', axis=alt.Axis(title=NEWCONFIRMED_CASES)),
    color=color,
    tooltip=[DATE_COLUMN, STATE_COLUMN, NEWCONFIRMED_CASES],
    order=alt.Order(
    STATE_COLUMN,
    sort='descending'
    )
).properties(
    title=CONFIRMED_CASES+": "+TOTAL_BY_STATE+' ('+NEW_CASES+')'
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(df).mark_point().encode(
    y=alt.Y(STATE_COLUMN+':N', axis=alt.Axis(orient='right')),
    color=color
).add_selection(
    selection1, selection2
)

plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) | legend
plot

alt.HConcatChart(...)

In [8]:
#hide_input
opt = df[STATE_COLUMN].unique()
opt.sort()
input_dropdown = alt.binding_select(options=opt)
selection1 = alt.selection_single(fields=[STATE_COLUMN], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=[STATE_COLUMN], on='mouseover')
color = alt.condition(selection1 | selection2,
                    alt.Color(STATE_COLUMN+':N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

chart = alt.Chart(df).mark_bar().encode(
    x=alt.X(DATE_COLUMN+':O', axis=alt.Axis(title=DATE_COLUMN)),
    y=alt.Y(NEWDEATH_COLUMN+':Q', axis=alt.Axis(title=NEWDEATH_COLUMN)),
    color=color,
    tooltip=[DATE_COLUMN, STATE_COLUMN, NEWDEATH_COLUMN],
    order=alt.Order(
    STATE_COLUMN,
    sort='descending'
    )
).properties(
    title=DEATHS+": "+TOTAL_BY_STATE+' ('+NEW_CASES+')'
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(df).mark_point().encode(
    y=alt.Y(STATE_COLUMN+':N', axis=alt.Axis(orient='right')),
    color=color
).add_selection(
    selection1, selection2
)

chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) | legend

alt.HConcatChart(...)

In [9]:
#hide_input
HTML(f'<small class="float-right">Última atualização em {pd.to_datetime(LAST_DATE).strftime("%d/%m/%Y")}</small>')

Based on works developed by [Alonso Silva Allende](https://covid19dashboards.com/jupyter/2020/04/27/Covid-19-Overview-Chile.html), adapted by [Cleber Jorge Amaral](http://cleberjamaral.github.io/). Data source: [brasil.io](https://brasil.io/home/)

In [10]:
#hide
save(plot,"../images/brazil-states-linear-cases-and-deaths-newcases.png")

ValueError: No enabled saver found that supports format='png'